In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn import svm
from sklearn.preprocessing import OneHotEncoder, StandardScaler
%matplotlib inline 
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import KMeansSMOTE
from imblearn.over_sampling import SMOTENC
from collections import Counter
import smote_variants as sv
from imblearn.over_sampling import SMOTE
from collections import Counter
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler

In [49]:
#data = pd.ExcelFile()
data = pd.read_excel('Earthquake Damage Data.xlsx', 'Data')
data.head()

,ID,P1,P2,P3,P4,P5,P6,BC1,BC2,BC3,BC4,BC5,BC6,BC7,BC8,DS
0,1,0.733800,0.588313,0.824660,0.420993,0.00,0.599542,0.257812,0,0.0,0,0.8,1.0,0.000000,0.827586,4
1,2,0.738113,0.604497,0.477239,0.252442,0.75,0.544622,0.148438,0,0.0,1,0.8,1.0,0.666667,0.931034,1
2,3,0.742815,0.602485,0.755395,0.434462,0.75,0.560641,0.093750,1,1.0,1,0.8,1.0,0.000000,0.655172,1
3,4,0.739635,0.602389,0.768629,0.484940,0.75,0.556064,0.148438,0,0.0,0,0.8,1.0,0.000000,0.965517,1
4,5,0.740052,0.605121,0.783227,0.486592,0.75,0.542334,0.148438,0,0.0,1,0.8,0.5,0.666667,0.931034,1


In [50]:
encode_array = OneHotEncoder()
x = data.iloc[:,[1,2,3,4,5,6,7,8,9,10,11,12,13,14]].values
y = data.iloc[:,15].values

print (x.shape)
print (y.shape)

(30590, 14)
(30590,)


In [51]:

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
)
# 获取维度参数
num_features = x_train.shape[1]  # 特征数量（14）
num_classes = len(np.unique(y))  # 类别数量（3）


In [52]:
smote = SMOTE(
    sampling_strategy={2: 4000, 3: 5000, 4: 15000},
    random_state=42
)
x_resampled, y_resampled = smote.fit_resample(x_train, y_train)
print("过采样后分布:", Counter(y_resampled))

过采样后分布: Counter({1: 22156, 4: 15000, 3: 5000, 2: 4000})


In [53]:
oversampler_0= sv.MulticlassOversampling('SMOTE')
x_train_resampled_0, y_train_resampled_0 = oversampler_0.sample(x_train, y_train)
oversampler_1= sv.MulticlassOversampling('distance_SMOTE')
x_train_resampled_1, y_train_resampled_1 = oversampler_1.sample(x_train, y_train)
# oversampler_2= under...('KMeans_SMOTE')   
#Undersampling
pipeline = Pipeline([
    ('oversample', SMOTE(sampling_strategy={2: 3000, 3: 3000, 4: 3000})),  # Oversample minorities
    ('undersample', RandomUnderSampler(sampling_strategy={1: 3000}))          # Undersample class 1
])

x_trian_resampled_2, y_trian_resampled_2 = pipeline.fit_resample(x_train, y_train)

2025-04-21 16:23:35,054:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2025-04-21 16:23:35,061:INFO:MulticlassOversampling: Sampling minority class with label: 4
2025-04-21 16:23:35,061:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2025-04-21 16:23:35,061:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2025-04-21 16:23:35,066:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2025-04-21 16:23:35,092:INFO:SMOTE: simplex sampling with n_dim 2
2025-04-21 16:23:35,133:INFO:MulticlassOversampling: Sampling minority class with label: 3
2025-04-21 16:23:35,134:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 0.498347144862564, 'n_neighbors': 5, 'nn_params': {}, 'n_

## KNN

In [54]:
model_KNN = KNeighborsClassifier(n_neighbors = 2)
model_KNN.fit(x_train,y_train)

KNeighborsClassifier(n_neighbors=2)

In [55]:
y_pred_KNN = model_KNN.predict(x_test)

In [56]:
print("\nClassification Report:\n", classification_report(y_test, y_pred_KNN))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_KNN))


Classification Report:
               precision    recall  f1-score   support

           1       0.91      0.98      0.95      5539
           2       0.27      0.12      0.17        24
           3       0.00      0.00      0.00        37
           4       0.35      0.09      0.14       518

    accuracy                           0.90      6118
   macro avg       0.38      0.30      0.31      6118
weighted avg       0.86      0.90      0.87      6118


Confusion Matrix:
 [[5438    8    7   86]
 [  21    3    0    0]
 [  36    0    0    1]
 [ 470    0    2   46]]


In [57]:
model_KNN.fit(x_resampled, y_resampled)
pred_0 = model_KNN.predict(x_test)

# 打印分类报告
print(metrics.classification_report(y_test, pred_0))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_0))

              precision    recall  f1-score   support

           1       0.92      0.88      0.90      5539
           2       0.08      0.21      0.11        24
           3       0.02      0.05      0.03        37
           4       0.22      0.28      0.24       518

    accuracy                           0.82      6118
   macro avg       0.31      0.36      0.32      6118
weighted avg       0.85      0.82      0.84      6118


Confusion Matrix:
 [[4890   58   79  512]
 [  17    5    2    0]
 [  33    0    2    2]
 [ 365    1    9  143]]


In [58]:
model_KNN.fit(x_train_resampled_0, y_train_resampled_0)
pred_1 = model_KNN.predict(x_test)  
        # 打印分类报告
print(metrics.classification_report(y_test, pred_1))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_1))

              precision    recall  f1-score   support

           1       0.92      0.87      0.90      5539
           2       0.06      0.17      0.09        24
           3       0.02      0.05      0.03        37
           4       0.22      0.31      0.26       518

    accuracy                           0.81      6118
   macro avg       0.30      0.35      0.32      6118
weighted avg       0.86      0.81      0.83      6118


Confusion Matrix:
 [[4818   64  103  554]
 [  17    4    3    0]
 [  31    0    2    4]
 [ 350    1    9  158]]


In [59]:
model_KNN.fit(x_train_resampled_1, y_train_resampled_1)
pred_2 = model_KNN.predict(x_test)  
# 打印分类报告
print(metrics.classification_report(y_test, pred_2))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_2))

              precision    recall  f1-score   support

           1       0.92      0.89      0.91      5539
           2       0.08      0.17      0.11        24
           3       0.03      0.05      0.04        37
           4       0.23      0.29      0.26       518

    accuracy                           0.83      6118
   macro avg       0.32      0.35      0.33      6118
weighted avg       0.86      0.83      0.84      6118


Confusion Matrix:
 [[4943   45   55  496]
 [  18    4    2    0]
 [  30    0    2    5]
 [ 363    1    5  149]]


In [60]:
model_KNN.fit(x_trian_resampled_2, y_trian_resampled_2)
pred_3 = model_KNN.predict(x_test)
# 打印分类告
print(metrics.classification_report(y_test, pred_3))
# 打印混淆矩阵
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_3))

              precision    recall  f1-score   support

           1       0.93      0.72      0.81      5539
           2       0.05      0.42      0.09        24
           3       0.02      0.24      0.04        37
           4       0.18      0.43      0.26       518

    accuracy                           0.69      6118
   macro avg       0.30      0.45      0.30      6118
weighted avg       0.86      0.69      0.76      6118


Confusion Matrix:
 [[3991  190  370  988]
 [  11   10    3    0]
 [  20    0    9    8]
 [ 258    2   36  222]]


# SVM

In [61]:
model_SVM = svm.SVC(kernel='linear')
model_SVM.fit(x_train, y_train)

SVC(kernel='linear')

In [62]:
y_pred_SVM = model_SVM.predict(x_test)

print("\nClassification Report:\n", classification_report(y_test, y_pred_SVM))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_SVM))


Classification Report:
               precision    recall  f1-score   support

           1       0.91      1.00      0.95      5539
           2       0.00      0.00      0.00        24
           3       0.00      0.00      0.00        37
           4       0.00      0.00      0.00       518

    accuracy                           0.91      6118
   macro avg       0.23      0.25      0.24      6118
weighted avg       0.82      0.91      0.86      6118


Confusion Matrix:
 [[5539    0    0    0]
 [  24    0    0    0]
 [  37    0    0    0]
 [ 518    0    0    0]]


d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [63]:
model_SVM.fit(x_resampled, y_resampled)
pred_0 = model_SVM.predict(x_test)

# 打印分类报告
print(metrics.classification_report(y_test, pred_0))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_0))

              precision    recall  f1-score   support

           1       0.92      0.75      0.83      5539
           2       0.04      0.71      0.08        24
           3       0.00      0.00      0.00        37
           4       0.16      0.37      0.22       518

    accuracy                           0.72      6118
   macro avg       0.28      0.46      0.28      6118
weighted avg       0.85      0.72      0.77      6118


Confusion Matrix:
 [[4178  357    0 1004]
 [   7   17    0    0]
 [  27    1    0    9]
 [ 316   11    0  191]]


d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [64]:
model_SVM.fit(x_train_resampled_0, y_train_resampled_0)
pred_1 = model_SVM.predict(x_test)  
# 打印分类报告
print(metrics.classification_report(y_test, pred_1))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_1))

              precision    recall  f1-score   support

           1       0.93      0.39      0.55      5539
           2       0.02      0.88      0.05        24
           3       0.01      0.43      0.02        37
           4       0.14      0.41      0.21       518

    accuracy                           0.40      6118
   macro avg       0.28      0.53      0.21      6118
weighted avg       0.85      0.40      0.52      6118


Confusion Matrix:
 [[2169  837 1268 1265]
 [   3   21    0    0]
 [   9    1   16   11]
 [ 156   24  124  214]]


In [65]:
model_SVM.fit(x_train_resampled_1, y_train_resampled_1)
pred_2 = model_SVM.predict(x_test)  
# 打印分类报告
print(metrics.classification_report(y_test, pred_2))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_2))

              precision    recall  f1-score   support

           1       0.93      0.39      0.54      5539
           2       0.02      0.83      0.04        24
           3       0.01      0.43      0.02        37
           4       0.15      0.41      0.22       518

    accuracy                           0.39      6118
   macro avg       0.28      0.52      0.21      6118
weighted avg       0.85      0.39      0.51      6118


Confusion Matrix:
 [[2137  840 1323 1239]
 [   4   20    0    0]
 [   9    1   16   11]
 [ 154   24  127  213]]


In [66]:
model_SVM.fit(x_trian_resampled_2, y_trian_resampled_2)
pred_3 = model_SVM.predict(x_test)

# 打印分类报告
print(metrics.classification_report(y_test, pred_3))        
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_3))


              precision    recall  f1-score   support

           1       0.92      0.37      0.53      5539
           2       0.02      0.88      0.04        24
           3       0.01      0.51      0.02        37
           4       0.13      0.36      0.19       518

    accuracy                           0.37      6118
   macro avg       0.27      0.53      0.20      6118
weighted avg       0.85      0.37      0.50      6118


Confusion Matrix:
 [[2065  866 1384 1224]
 [   3   21    0    0]
 [   9    1   19    8]
 [ 156   23  155  184]]


# linear regression

In [67]:
model_lr = LinearRegression()
model_lr.fit(x_train, y_train)
y_pred_lr = model_lr.predict(x_test)
print("预测值示例:", y_pred_lr[:5])


预测值示例: [1.05470751 1.17007976 1.49089912 1.21049263 1.33937797]


#### 预测值是分数（连续数值），而不是离散的类别标签 ​线性回归模型（LinearRegression）​，而线性回归是专门用于解决 ​回归任务 的模型（预测连续值）

# logistic regression

In [68]:
from sklearn.linear_model import LogisticRegression
model_lg = LogisticRegression(multi_class='multinomial', max_iter=1000)
model_lg.fit(x_train, y_train)
y_pred_lg = model_lg.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred_lg))
print("\nClassification Report:\n", classification_report(y_test, y_pred_lg))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_lg))

d:\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.9053612291598562

Classification Report:
               precision    recall  f1-score   support

           1       0.91      1.00      0.95      5539
           2       0.00      0.00      0.00        24
           3       0.00      0.00      0.00        37
           4       0.00      0.00      0.00       518

    accuracy                           0.91      6118
   macro avg       0.23      0.25      0.24      6118
weighted avg       0.82      0.91      0.86      6118


Confusion Matrix:
 [[5539    0    0    0]
 [  24    0    0    0]
 [  37    0    0    0]
 [ 518    0    0    0]]


d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [69]:
model_lg.fit(x_resampled, y_resampled)
pred_0 = model_lg.predict(x_test)

# 打印分类报告
print(metrics.classification_report(y_test, pred_0))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_0))

d:\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


              precision    recall  f1-score   support

           1       0.92      0.75      0.83      5539
           2       0.02      0.38      0.05        24
           3       0.00      0.00      0.00        37
           4       0.15      0.36      0.22       518

    accuracy                           0.71      6118
   macro avg       0.27      0.37      0.27      6118
weighted avg       0.85      0.71      0.77      6118


Confusion Matrix:
 [[4145  352    8 1034]
 [  15    9    0    0]
 [  26    0    0   11]
 [ 319   10    0  189]]


In [70]:
model_lg.fit(x_train_resampled_0, y_train_resampled_0)
pred_1 = model_lg.predict(x_test)  
        # 打印分类报告
print(metrics.classification_report(y_test, pred_1))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_1))

d:\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


              precision    recall  f1-score   support

           1       0.94      0.32      0.48      5539
           2       0.02      0.92      0.04        24
           3       0.01      0.35      0.02        37
           4       0.13      0.44      0.20       518

    accuracy                           0.33      6118
   macro avg       0.28      0.51      0.19      6118
weighted avg       0.86      0.33      0.45      6118


Confusion Matrix:
 [[1769  924 1346 1500]
 [   1   22    0    1]
 [   9    1   13   14]
 [ 105   26  157  230]]


In [71]:
model_lg.fit(x_train_resampled_1, y_train_resampled_1)
pred_2 = model_lg.predict(x_test)  
# 打印分类报告
print(metrics.classification_report(y_test, pred_2))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_2))

d:\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


              precision    recall  f1-score   support

           1       0.94      0.32      0.48      5539
           2       0.02      0.92      0.04        24
           3       0.01      0.35      0.02        37
           4       0.14      0.45      0.21       518

    accuracy                           0.34      6118
   macro avg       0.28      0.51      0.19      6118
weighted avg       0.86      0.34      0.45      6118


Confusion Matrix:
 [[1783  924 1387 1445]
 [   1   22    0    1]
 [   9    1   13   14]
 [ 103   26  155  234]]


In [72]:
model_lg.fit(x_trian_resampled_2, y_trian_resampled_2)
pred_3 = model_lg.predict(x_test)

# 打印分类报告
print(metrics.classification_report(y_test, pred_3))        
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_3))

d:\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


              precision    recall  f1-score   support

           1       0.94      0.32      0.48      5539
           2       0.02      0.92      0.04        24
           3       0.01      0.38      0.02        37
           4       0.13      0.44      0.20       518

    accuracy                           0.33      6118
   macro avg       0.27      0.51      0.18      6118
weighted avg       0.86      0.33      0.45      6118


Confusion Matrix:
 [[1786  984 1257 1512]
 [   1   22    0    1]
 [   9    1   14   13]
 [ 112   44  135  227]]


# Naive bayes

In [73]:
from sklearn.naive_bayes import GaussianNB
model_nv = GaussianNB()
model_nv.fit(x_train, y_train)
y_pred_nv = model_nv.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred_nv))
print("\nClassification Report:\n", classification_report(y_test, y_pred_nv))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_nv))

Accuracy: 0.4818568159529258

Classification Report:
               precision    recall  f1-score   support

           1       0.91      0.50      0.65      5539
           2       0.01      0.96      0.02        24
           3       0.00      0.00      0.00        37
           4       0.14      0.27      0.19       518

    accuracy                           0.48      6118
   macro avg       0.27      0.43      0.21      6118
weighted avg       0.84      0.48      0.60      6118


Confusion Matrix:
 [[2784 1922    5  828]
 [   1   23    0    0]
 [  18    5    0   14]
 [ 252  125    0  141]]


In [74]:
model_nv.fit(x_resampled, y_resampled)
pred_0 = model_nv.predict(x_test)

# 打印分类报告
print(metrics.classification_report(y_test, pred_0))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_0))

              precision    recall  f1-score   support

           1       0.93      0.27      0.42      5539
           2       0.01      0.92      0.02        24
           3       0.03      0.16      0.05        37
           4       0.12      0.52      0.19       518

    accuracy                           0.30      6118
   macro avg       0.27      0.47      0.17      6118
weighted avg       0.85      0.30      0.40      6118


Confusion Matrix:
 [[1523 1861  181 1974]
 [   2   22    0    0]
 [   6    4    6   21]
 [ 107  127   13  271]]


In [75]:
model_nv.fit(x_train_resampled_0, y_train_resampled_0)
pred_1 = model_nv.predict(x_test)  
        # 打印分类报告
print(metrics.classification_report(y_test, pred_1))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_1))

              precision    recall  f1-score   support

           1       0.94      0.22      0.35      5539
           2       0.01      0.96      0.02        24
           3       0.01      0.27      0.03        37
           4       0.12      0.50      0.20       518

    accuracy                           0.24      6118
   macro avg       0.27      0.49      0.15      6118
weighted avg       0.86      0.24      0.34      6118


Confusion Matrix:
 [[1207 1879  646 1807]
 [   1   23    0    0]
 [   3    4   10   20]
 [  79  127   55  257]]


In [76]:
model_nv.fit(x_train_resampled_1, y_train_resampled_1)
pred_2 = model_nv.predict(x_test)  
# 打印分类报告
print(metrics.classification_report(y_test, pred_2))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_2))

              precision    recall  f1-score   support

           1       0.94      0.23      0.38      5539
           2       0.01      0.92      0.02        24
           3       0.01      0.32      0.02        37
           4       0.13      0.44      0.20       518

    accuracy                           0.26      6118
   macro avg       0.27      0.48      0.16      6118
weighted avg       0.86      0.26      0.36      6118


Confusion Matrix:
 [[1298 1845  874 1522]
 [   2   22    0    0]
 [   4    4   12   17]
 [  78  125   86  229]]


In [77]:
model_nv.fit(x_trian_resampled_2, y_trian_resampled_2)
pred_3 = model_nv.predict(x_test)
# 打印分类报告
print(metrics.classification_report(y_test, pred_3))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_3))


              precision    recall  f1-score   support

           1       0.94      0.21      0.34      5539
           2       0.01      0.96      0.02        24
           3       0.01      0.27      0.02        37
           4       0.12      0.49      0.20       518

    accuracy                           0.23      6118
   macro avg       0.27      0.48      0.15      6118
weighted avg       0.86      0.23      0.32      6118


Confusion Matrix:
 [[1150 1879  723 1787]
 [   1   23    0    0]
 [   3    4   10   20]
 [  66  128   70  254]]


# Decision tree

In [78]:
from sklearn.tree import DecisionTreeClassifier
model_DT = DecisionTreeClassifier(
    random_state=42,
    class_weight='balanced'  # 自动平衡类别权重
)
model_DT.fit(x_train, y_train)
y_pred_DT = model_DT.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred_DT))
print("\nClassification Report:\n", classification_report(y_test, y_pred_DT))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_DT))

Accuracy: 0.8592677345537757

Classification Report:
               precision    recall  f1-score   support

           1       0.92      0.92      0.92      5539
           2       0.05      0.04      0.04        24
           3       0.03      0.03      0.03        37
           4       0.26      0.27      0.27       518

    accuracy                           0.86      6118
   macro avg       0.32      0.31      0.32      6118
weighted avg       0.86      0.86      0.86      6118


Confusion Matrix:
 [[5117   20   26  376]
 [  18    1    0    5]
 [  33    0    1    3]
 [ 377    0    3  138]]


In [79]:
model_DT.fit(x_resampled, y_resampled)
pred_0 = model_DT.predict(x_test)

# 打印分类报告
print(metrics.classification_report(y_test, pred_0))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_0))

              precision    recall  f1-score   support

           1       0.93      0.85      0.89      5539
           2       0.05      0.12      0.07        24
           3       0.04      0.11      0.06        37
           4       0.23      0.37      0.28       518

    accuracy                           0.81      6118
   macro avg       0.31      0.37      0.32      6118
weighted avg       0.86      0.81      0.83      6118


Confusion Matrix:
 [[4729   59   92  659]
 [  18    3    0    3]
 [  30    0    4    3]
 [ 312    0   12  194]]


In [80]:
model_DT.fit(x_train_resampled_0, y_train_resampled_0)
pred_1 = model_DT.predict(x_test)  
        # 打印分类报告
print(metrics.classification_report(y_test, pred_1))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_1))

              precision    recall  f1-score   support

           1       0.93      0.85      0.89      5539
           2       0.07      0.17      0.10        24
           3       0.02      0.08      0.04        37
           4       0.21      0.37      0.27       518

    accuracy                           0.80      6118
   macro avg       0.31      0.37      0.32      6118
weighted avg       0.86      0.80      0.83      6118


Confusion Matrix:
 [[4693   49  111  686]
 [  16    4    1    3]
 [  28    0    3    6]
 [ 317    1   10  190]]


In [81]:
model_DT.fit(x_train_resampled_1, y_train_resampled_1)
pred_2 = model_DT.predict(x_test)  
# 打印分类报告
print(metrics.classification_report(y_test, pred_2))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_2))

              precision    recall  f1-score   support

           1       0.93      0.90      0.91      5539
           2       0.08      0.12      0.10        24
           3       0.04      0.05      0.04        37
           4       0.25      0.31      0.28       518

    accuracy                           0.84      6118
   macro avg       0.32      0.35      0.33      6118
weighted avg       0.86      0.84      0.85      6118


Confusion Matrix:
 [[4978   34   47  480]
 [  16    3    2    3]
 [  33    0    2    2]
 [ 350    0    6  162]]


In [82]:
model_DT.fit(x_trian_resampled_2, y_trian_resampled_2)
pred_3 = model_DT.predict(x_test)

# 打印分类报告
print(metrics.classification_report(y_test, pred_3))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_3))

              precision    recall  f1-score   support

           1       0.95      0.64      0.76      5539
           2       0.05      0.38      0.09        24
           3       0.02      0.24      0.04        37
           4       0.18      0.64      0.28       518

    accuracy                           0.64      6118
   macro avg       0.30      0.47      0.29      6118
weighted avg       0.87      0.64      0.72      6118


Confusion Matrix:
 [[3545  163  335 1496]
 [  10    9    0    5]
 [  16    0    9   12]
 [ 165    2   21  330]]


# ANN(MLP)

In [83]:
from sklearn.neural_network import MLPClassifier
model_MLP = MLPClassifier(
    hidden_layer_sizes=(50, 20),  # 隐藏层结构
    activation='relu',
    solver='adam',
    max_iter=200,
    random_state=42,
    
   
)


model_MLP.fit(x_train, y_train)
y_pred_MLP = model_MLP.predict(x_test)
print("\nClassification Report:\n", classification_report(y_test, y_pred_MLP))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_MLP))


Classification Report:
               precision    recall  f1-score   support

           1       0.91      1.00      0.95      5539
           2       0.00      0.00      0.00        24
           3       0.00      0.00      0.00        37
           4       0.00      0.00      0.00       518

    accuracy                           0.90      6118
   macro avg       0.23      0.25      0.24      6118
weighted avg       0.82      0.90      0.86      6118


Confusion Matrix:
 [[5531    0    0    8]
 [  24    0    0    0]
 [  37    0    0    0]
 [ 518    0    0    0]]


d:\anaconda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicte

In [84]:
model_MLP.fit(x_resampled, y_resampled)
pred_0 = model_MLP.predict(x_test)

# 打印分类报告
print(metrics.classification_report(y_test, pred_0))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_0))

              precision    recall  f1-score   support

           1       0.94      0.71      0.81      5539
           2       0.07      0.67      0.12        24
           3       0.02      0.16      0.04        37
           4       0.19      0.51      0.28       518

    accuracy                           0.69      6118
   macro avg       0.31      0.51      0.31      6118
weighted avg       0.87      0.69      0.76      6118


Confusion Matrix:
 [[3959  213  251 1116]
 [   7   16    1    0]
 [  21    0    6   10]
 [ 233    3   16  266]]


d:\anaconda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [85]:
model_MLP.fit(x_train_resampled_0, y_train_resampled_0)
pred_1 = model_MLP.predict(x_test)  
        # 打印分类报告
print(metrics.classification_report(y_test, pred_1))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_1))

              precision    recall  f1-score   support

           1       0.95      0.57      0.71      5539
           2       0.04      0.67      0.08        24
           3       0.02      0.27      0.04        37
           4       0.17      0.61      0.26       518

    accuracy                           0.57      6118
   macro avg       0.29      0.53      0.27      6118
weighted avg       0.87      0.57      0.67      6118


Confusion Matrix:
 [[3170  338  465 1566]
 [   6   16    1    1]
 [  18    0   10    9]
 [ 147    9   47  315]]


d:\anaconda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [86]:
model_MLP.fit(x_train_resampled_1, y_train_resampled_1)
pred_2 = model_MLP.predict(x_test)  
# 打印分类报告
print(metrics.classification_report(y_test, pred_2))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_2))

              precision    recall  f1-score   support

           1       0.95      0.57      0.71      5539
           2       0.04      0.54      0.08        24
           3       0.02      0.30      0.03        37
           4       0.18      0.62      0.27       518

    accuracy                           0.57      6118
   macro avg       0.30      0.51      0.27      6118
weighted avg       0.88      0.57      0.67      6118


Confusion Matrix:
 [[3154  278  610 1497]
 [   8   13    3    0]
 [  14    0   11   12]
 [ 136    5   54  323]]


d:\anaconda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [87]:
model_MLP.fit(x_trian_resampled_2, y_trian_resampled_2)
pred_3 = model_MLP.predict(x_test)
# 打印分类报告
print(metrics.classification_report(y_test, pred_3))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred_3))

              precision    recall  f1-score   support

           1       0.95      0.44      0.60      5539
           2       0.03      0.71      0.07        24
           3       0.02      0.46      0.03        37
           4       0.15      0.62      0.24       518

    accuracy                           0.45      6118
   macro avg       0.29      0.56      0.24      6118
weighted avg       0.87      0.45      0.56      6118


Confusion Matrix:
 [[2423  471  883 1762]
 [   5   17    1    1]
 [  10    0   17   10]
 [ 112   10   77  319]]


d:\anaconda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


# CNN

In [88]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# 构建 1D-CNN 模型
model_CNN = models.Sequential([
    layers.Conv1D(32, kernel_size=3, activation='relu', 
                  kernel_regularizer=regularizers.l2(0.001),
                  input_shape=(14, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.3),
    layers.Conv1D(64, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model_CNN.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 训练
history = model_CNN.fit(
    x_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)
model_CNN.summary()
# 预测与评估
y_pred_CNN = model_CNN.predict(x_test)
y_pred_CNN = np.argmax(y_pred_CNN, axis=1)

print("Accuracy:", accuracy_score(y_test, y_pred_CNN))
print("\nClassification Report:\n", classification_report(y_test, y_pred_CNN))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_CNN))

Epoch 1/10


d:\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


612/612 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8274 - loss: 0.6890 - val_accuracy: 0.9017 - val_loss: 0.3757
Epoch 2/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9117 - loss: 0.3571 - val_accuracy: 0.9015 - val_loss: 0.3486
Epoch 3/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9063 - loss: 0.3629 - val_accuracy: 0.9015 - val_loss: 0.3409
Epoch 4/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9037 - loss: 0.3554 - val_accuracy: 0.9015 - val_loss: 0.3363
Epoch 5/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9089 - loss: 0.3393 - val_accuracy: 0.9015 - val_loss: 0.3337
Epoch 6/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9074 - loss: 0.3450 - val_accuracy: 0.9015 - val_loss: 0.3312
Epoch 7/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9084 - loss: 0.3308 - val_accuracy: 0.9015 - val_loss: 0.3302
Epoch 8/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9061 - loss: 0.3347 - val_accuracy: 0.9015 - val_

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 12, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 12, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 6, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 6, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 4, 64)          │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 4, 64)          │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 2, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Trainable params: 15,434 (60.29 KB)

 Non-trainable params: 192 (768.00 B)

 Optimizer params: 30,870 (120.59 KB)

192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.9053612291598562

Classification Report:
               precision    recall  f1-score   support

           1       0.91      1.00      0.95      5539
           2       0.00      0.00      0.00        24
           3       0.00      0.00      0.00        37
           4       0.00      0.00      0.00       518

    accuracy                           0.91      6118
   macro avg       0.23      0.25      0.24      6118
weighted avg       0.82      0.91      0.86      6118


Confusion Matrix:
 [[5539    0    0    0]
 [  24    0    0    0]
 [  37    0    0    0]
 [ 518    0    0    0]]


d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [89]:


model_CNN.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 训练
history = model_CNN.fit(
    x_resampled, y_resampled,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)
y_pred_CNN = model_CNN.predict(x_test)
y_pred_CNN = np.argmax(y_pred_CNN, axis=1)


print("\nClassification Report:\n", classification_report(y_test, y_pred_CNN))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_CNN))

Epoch 1/10
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6098 - loss: 0.9955 - val_accuracy: 0.1196 - val_loss: 1.4411
Epoch 2/10
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6308 - loss: 0.8633 - val_accuracy: 0.1036 - val_loss: 1.5922
Epoch 3/10
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6383 - loss: 0.8362 - val_accuracy: 0.1239 - val_loss: 1.4343
Epoch 4/10
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6462 - loss: 0.8263 - val_accuracy: 0.1457 - val_loss: 1.3912
Epoch 5/10
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6515 - loss: 0.8076 - val_accuracy: 0.1108 - val_loss: 1.3429
Epoch 6/10
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6596 - loss: 0.7870 - val_accuracy: 0.1118 - val_loss: 1.5516
Epoch 7/10
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6623 - loss: 0.7833 - val_accuracy: 0.0898 - val_loss: 1.5008
Epoch 8/10
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6659 - loss: 0.7735 - 

In [90]:
model_CNN.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 训练
history = model_CNN.fit(
    x_train_resampled_0, y_train_resampled_0,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)
y_pred_CNN = model_CNN.predict(x_test)
y_pred_CNN = np.argmax(y_pred_CNN, axis=1)

print("Accuracy:", accuracy_score(y_test, y_pred_CNN))
print("\nClassification Report:\n", classification_report(y_test, y_pred_CNN))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_CNN))

Epoch 1/10


2216/2216 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.5825 - loss: 0.8852 - val_accuracy: 0.8705 - val_loss: 0.5465
Epoch 2/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.6093 - loss: 0.8512 - val_accuracy: 0.9155 - val_loss: 0.5264
Epoch 3/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.6133 - loss: 0.8452 - val_accuracy: 0.9021 - val_loss: 0.5565
Epoch 4/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.6248 - loss: 0.8235 - val_accuracy: 0.9184 - val_loss: 0.4928
Epoch 5/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.6262 - loss: 0.8204 - val_accuracy: 0.9375 - val_loss: 0.4790
Epoch 6/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.6251 - loss: 0.8184 - val_accuracy: 0.9542 - val_loss: 0.4917
Epoch 7/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.6310 - loss: 0.8043 - val_accuracy: 0.9352 - val_loss: 0.4989
Epoch 8/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.6368 - loss: 0.8006 - val_accurac

In [91]:
model_CNN.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 训练
history = model_CNN.fit(
    x_train_resampled_1, y_train_resampled_1,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)
y_pred_CNN = model_CNN.predict(x_test)
y_pred_CNN = np.argmax(y_pred_CNN, axis=1)

print("Accuracy:", accuracy_score(y_test, y_pred_CNN))
print("\nClassification Report:\n", classification_report(y_test, y_pred_CNN))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_CNN))

Epoch 1/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.6653 - loss: 0.7477 - val_accuracy: 0.9909 - val_loss: 0.3856
Epoch 2/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.6674 - loss: 0.7388 - val_accuracy: 0.9771 - val_loss: 0.3881
Epoch 3/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.6714 - loss: 0.7294 - val_accuracy: 0.9475 - val_loss: 0.3180
Epoch 4/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.6788 - loss: 0.7253 - val_accuracy: 0.9919 - val_loss: 0.3462
Epoch 5/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.6825 - loss: 0.7191 - val_accuracy: 0.9935 - val_loss: 0.3389
Epoch 6/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.6806 - loss: 0.7152 - val_accuracy: 0.9337 - val_loss: 0.4010
Epoch 7/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.6846 - loss: 0.7150 - val_accuracy: 0.9875 - val_loss: 0.2884
Epoch 8/10
2216/2216 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.6878 - loss: 0.7041 - 

In [92]:
history = model_CNN.fit(
    x_trian_resampled_2, y_trian_resampled_2,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)
y_pred_CNN = model_CNN.predict(x_test)
y_pred_CNN = np.argmax(y_pred_CNN, axis=1)
print("Accuracy:", accuracy_score(y_test, y_pred_CNN))
print("\nClassification Report:\n", classification_report(y_test, y_pred_CNN))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_CNN))

Epoch 1/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7657 - loss: 0.6224 - val_accuracy: 0.0942 - val_loss: 1.7033
Epoch 2/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7906 - loss: 0.5697 - val_accuracy: 0.0796 - val_loss: 1.7733
Epoch 3/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7946 - loss: 0.5715 - val_accuracy: 0.0246 - val_loss: 1.8101
Epoch 4/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7941 - loss: 0.5596 - val_accuracy: 0.0767 - val_loss: 1.8025
Epoch 5/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7921 - loss: 0.5585 - val_accuracy: 0.0887 - val_loss: 1.7140
Epoch 6/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7952 - loss: 0.5471 - val_accuracy: 0.0217 - val_loss: 1.8058
Epoch 7/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8029 - loss: 0.5445 - val_accuracy: 0.0137 - val_loss: 1.8328
Epoch 8/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7961 - loss: 0.5549 - val_accuracy: 0.

# XGBOOST

In [93]:

from xgboost import XGBClassifier
model_XG = XGBClassifier()
model_XG.fit(x_train.squeeze(), y_train-1)
y_pred_XG = model_XG.predict(x_test.squeeze())


print("Accuracy:", accuracy_score(y_test-1, y_pred_XG))
print("\nClassification Report:\n", classification_report(y_test-1, y_pred_XG))
print("\nConfusion Matrix:\n", confusion_matrix(y_test-1, y_pred_XG))

Accuracy: 0.905197777051324

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.99      0.95      5539
           1       0.00      0.00      0.00        24
           2       0.50      0.03      0.05        37
           3       0.50      0.11      0.18       518

    accuracy                           0.91      6118
   macro avg       0.48      0.28      0.29      6118
weighted avg       0.87      0.91      0.88      6118


Confusion Matrix:
 [[5481    1    1   56]
 [  24    0    0    0]
 [  36    0    1    0]
 [ 462    0    0   56]]


In [94]:
model_XG.fit(x_resampled, y_resampled-1)
y_pred_XG = model_XG.predict(x_test.squeeze())

# 打印分类报告
print(metrics.classification_report(y_test-1, y_pred_XG))
print("\nConfusion Matrix:\n", confusion_matrix(y_test-1, y_pred_XG))

              precision    recall  f1-score   support

           0       0.93      0.90      0.92      5539
           1       0.09      0.21      0.12        24
           2       0.05      0.03      0.03        37
           3       0.28      0.35      0.31       518

    accuracy                           0.85      6118
   macro avg       0.33      0.37      0.35      6118
weighted avg       0.86      0.85      0.86      6118


Confusion Matrix:
 [[5008   51   19  461]
 [  19    5    0    0]
 [  33    0    1    3]
 [ 336    1    2  179]]


In [95]:
model_XG.fit(x_train_resampled_0, y_train_resampled_0-1)
y_pred_XG = model_XG.predict(x_test.squeeze())

# 打印分类报告
print(metrics.classification_report(y_test-1, y_pred_XG))
print("\nConfusion Matrix:\n", confusion_matrix(y_test-1, y_pred_XG))

              precision    recall  f1-score   support

           0       0.93      0.87      0.90      5539
           1       0.08      0.29      0.12        24
           2       0.02      0.05      0.03        37
           3       0.28      0.42      0.34       518

    accuracy                           0.83      6118
   macro avg       0.33      0.41      0.35      6118
weighted avg       0.87      0.83      0.85      6118


Confusion Matrix:
 [[4828   83   77  551]
 [  16    7    1    0]
 [  32    0    2    3]
 [ 292    1    9  216]]


In [96]:
model_XG.fit(x_train_resampled_1, y_train_resampled_1-1)
y_pred_XG = model_XG.predict(x_test.squeeze())

# 打印分类报告
print(metrics.classification_report(y_test-1, y_pred_XG))
print("\nConfusion Matrix:\n", confusion_matrix(y_test-1, y_pred_XG))

              precision    recall  f1-score   support

           0       0.92      0.95      0.93      5539
           1       0.12      0.12      0.12        24
           2       0.20      0.03      0.05        37
           3       0.33      0.26      0.29       518

    accuracy                           0.88      6118
   macro avg       0.39      0.34      0.35      6118
weighted avg       0.87      0.88      0.87      6118


Confusion Matrix:
 [[5244   22    3  270]
 [  21    3    0    0]
 [  34    0    1    2]
 [ 383    1    1  133]]


In [97]:
model_XG.fit(x_trian_resampled_2, y_trian_resampled_2-1)
y_pred_XG = model_XG.predict(x_test.squeeze())  
# 打印分类报告

print(metrics.classification_report(y_test-1, y_pred_XG))
print("\nConfusion Matrix:\n", confusion_matrix(y_test-1, y_pred_XG))


              precision    recall  f1-score   support

           0       0.96      0.71      0.81      5539
           1       0.06      0.46      0.10        24
           2       0.04      0.16      0.06        37
           3       0.21      0.69      0.32       518

    accuracy                           0.70      6118
   macro avg       0.32      0.50      0.32      6118
weighted avg       0.88      0.70      0.76      6118


Confusion Matrix:
 [[3908  174  150 1307]
 [  10   11    2    1]
 [  19    0    6   12]
 [ 151    1   11  355]]
